In [1]:
from pyspark.sql import *

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

23/05/08 15:59:15 WARN Utils: Your hostname, MacBook-Pro-Sofi.local resolves to a loopback address: 127.0.0.1; using 109.123.185.23 instead (on interface en6)
23/05/08 15:59:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/08 15:59:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/08 15:59:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv('Datasets/FIFA.csv', sep=',', header=True, inferSchema=True, multiLine=True, escape='"')

In [4]:
data.show(n=100, truncate=False)

+-------------------+----+-------------------+-------------------+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+------------------------------------------------------------+--------------------------------------------+------------------------------+---------+-------+
|ID            

In [5]:
# Далее, данные преобразуются в RDD (распределенный набор данных) для дальнейшей работы с ними:
rdd = data.rdd

In [6]:
# выбираем пользователей, которых упоминали в твитах чаще всего
top_users = rdd.flatMap(lambda x: x.UserMentionID.lower().split(',') if x.UserMentionID is not None else [])
top_users = top_users.map(lambda x: (x, 1))
top_users = top_users.reduceByKey(lambda x, y: x + y)
top_users = top_users.top(5, key=lambda x: x[1])
print(top_users)

[('fifaworldcup', 91122), ('khaledbeydoun', 17507), ('weareoneexo', 14974), ('hns_cff', 11836), ('paulpogba', 11465)]


In [7]:
# для каждого из этих пользователей определяем десять наиболее часто употребляемых вместе с ними хештегов
users_tags = []
for user, _ in top_users:
    tweets_from_user = rdd.filter(lambda x: x.UserMentionID is not None)
    tweets_from_user = tweets_from_user.filter(lambda x: user in x.UserMentionID.lower().split(','))

    tags_from_user = tweets_from_user.flatMap(lambda x: x.Hashtags.lower().split(',') if x.Hashtags != None else [])
    tags_from_user = tags_from_user.map(lambda x: (x, 1))
    tags_from_user = tags_from_user.reduceByKey(lambda x, y: x + y)
    top_tags = tags_from_user.top(10, key=lambda x: x[1])

    users_tags.append((user, top_tags))
    print((user, top_tags))

('fifaworldcup', [('worldcup', 77249), ('fra', 24828), ('fifastadiumdj', 13563), ('worldcupfinal', 13551), ('fracro', 9147), ('cro', 8565), ('powerbyexo', 7325), ('bel', 6449), ('frabel', 5927), ('exo', 4679)])


('khaledbeydoun', [('worldcup', 17432), ('france', 76), ('belgium', 12), ('fra', 2), ('african', 1), ('sports', 1), ('arts', 1), ('music', 1), ('technology', 1), ('ideas', 1)])


('weareoneexo', [('worldcup', 11366), ('fifastadiumdj', 10516), ('powerbyexo', 7217), ('exo', 5037), ('??', 2910), ('dafbama2018exo', 1148), ('power', 585), ('copa2018', 547), ('chanyeolleague', 365), ('layleague', 358)])


('hns_cff', [('worldcup', 10094), ('cro', 8429), ('croeng', 2363), ('beproud', 1837), ('family', 1489), ('vatreni', 1472), ('croden', 1231), ('flamingpride', 886), ('croatia', 844), ('worldcupfinal', 788)])


('paulpogba', [('worldcup', 11327), ('fra', 2900), ('heretocreate', 2438), ('cro', 621), ('bel', 594), ('mufc', 187), ('fraarg', 129), ('worldcupfinal', 108), ('u18pl', 87), ('arg', 76)])


In [8]:
# выбираем пользователя, у которого хештеги наиболее отличаются от четырех остальных
tag_sets = []
for user, tags in users_tags:
    tag_sets.append(set(map(lambda x: x[0], tags)))

most_different_user = None
most_different_tags = None
most_different_score = 0

for i, tag_set in enumerate(tag_sets):
    score = 0
    for j, other_tag_set in enumerate(tag_sets):
        if i == j:
            continue
        score += len(tag_set.symmetric_difference(other_tag_set))
    if score > most_different_score:
        most_different_user = top_users[i][0]
        most_different_tags = tag_set
        most_different_score = score

print(most_different_user, most_different_tags)

khaledbeydoun {'technology', 'france', 'arts', 'belgium', 'ideas', 'worldcup', 'music', 'sports', 'african', 'fra'}


In [9]:
# выбираем два пользователя, у которых наиболее близкое друг к другу соотношение частоты встречаемости выделенных хештегов
user_tag_pairs = []
for user, tags in users_tags:
    tag_set = set(map(lambda x: x[0], tags))
    user_tag_pairs.append((user, tag_set))

closest_users = None
closest_sim = 0

for i, (user1, tag_set1) in enumerate(user_tag_pairs):
    for j in range(i + 1, len(user_tag_pairs)):
        user2, tag_set2 = user_tag_pairs[j]
        sim = len(tag_set1.intersection(tag_set2)) / len(tag_set1.union(tag_set2))
        if sim > closest_sim:
            closest_users = (user1, user2)
            closest_sim = sim

print(closest_users, closest_sim)

('fifaworldcup', 'paulpogba') 0.3333333333333333
